# Modelagem de CNN - Pneumonia em Raio-X

Este notebook implementa e treina modelos de Redes Neurais Convolucionais (CNNs) para classificação de pneumonia em imagens de raio-X de tórax.

## Metodologia

- **Pré-processamento**: Redimensionamento, normalização, data augmentation
- **Divisão dos Dados**: 60% treino / 20% validação / 20% teste
- **Modelos**: CNN construída do zero
- **Avaliação**: Accuracy, Precision, Recall, F1-Score, ROC-AUC
- **Interpretabilidade**: Grad-CAM para visualizar regiões importantes


In [ ]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
import tensorflow as tf
from tensorflow import keras

# Adicionar o diretório raiz do projeto ao sys.path
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))

# Importar módulos do projeto
from src.vision.data_loader import load_image_dataset
from src.vision.preprocessing import split_image_data, create_data_generators
from src.vision.models import (
    create_simple_cnn_pneumonia,
    compile_model,
    get_model_callbacks
)
from src.vision.evaluation import (
    plot_training_history,
    plot_confusion_matrix,
    plot_roc_curve,
    visualize_predictions,
    plot_grad_cam,
    evaluate_model
)

# Configuração
with open("../config.yaml", "r") as f:
    config = yaml.safe_load(f)

# Configurações do TensorFlow
tf.random.set_seed(config["split"]["random_state"])
np.random.seed(config["split"]["random_state"])

print("Módulos importados com sucesso!")
print(f"TensorFlow version: {tf.__version__}")


## 1. Preparação dos Dados

Carregamos o dataset e dividimos em conjuntos de treino, validação e teste.


In [ ]:
# Carregar dataset
# Assumindo que o dataset já foi baixado no notebook de exploração
# Se não, descomente a linha abaixo:
# from src.vision.data_loader import download_pneumonia_dataset
# dataset_path = download_pneumonia_dataset(config["data"]["images"]["pneumonia_path"])

# Para este exemplo, vamos assumir que temos o path do dataset
# Em produção, você pode salvar o path do notebook de exploração
dataset_path = config["data"]["images"]["pneumonia_path"]

# Carregar imagens
df = load_image_dataset(dataset_path)

print(f"Total de imagens: {len(df)}")
print(f"Classes: {df['label'].unique()}")

# Se o dataset já tem split (train/test), vamos usar
# Caso contrário, vamos criar nosso próprio split
if 'split' in df.columns and 'train' in df['split'].values:
    # Usar split existente
    train_df = df[df['split'] == 'train'].copy()
    test_df = df[df['split'] == 'test'].copy() if 'test' in df['split'].values else None
    
    # Se não houver validação, criar a partir do treino
    if 'val' in df['split'].values or 'validation' in df['split'].values:
        val_df = df[df['split'].isin(['val', 'validation'])].copy()
    else:
        train_df, val_df, _ = split_image_data(
            train_df,
            test_size=0.2,
            validation_size=0.2,
            random_state=config["split"]["random_state"]
        )
    
    if test_df is None:
        # Criar teste a partir do treino se não existir
        train_df, _, test_df = split_image_data(
            train_df,
            test_size=0.2,
            validation_size=0.0,
            random_state=config["split"]["random_state"]
        )
else:
    # Criar split do zero
    train_df, val_df, test_df = split_image_data(
        df,
        test_size=config["split"]["test_size"],
        validation_size=config["split"]["validation_size"],
        random_state=config["split"]["random_state"]
    )

print(f"\nDivisão dos dados:")
print(f"  Treino:    {len(train_df)} amostras ({len(train_df)/len(df)*100:.1f}%)")
print(f"  Validação: {len(val_df)} amostras ({len(val_df)/len(df)*100:.1f}%)")
print(f"  Teste:     {len(test_df)} amostras ({len(test_df)/len(df)*100:.1f}%)")

print(f"\nDistribuição das classes no treino:")
print(train_df['label'].value_counts())
print(f"\nDistribuição das classes na validação:")
print(val_df['label'].value_counts())
print(f"\nDistribuição das classes no teste:")
print(test_df['label'].value_counts())


### 1.1 Criar Data Generators

Criamos generators para treino, validação e teste com data augmentation para o conjunto de treino.


In [ ]:
# Configurações
image_size = tuple(config["models"]["cnn"]["image_size"]["pneumonia"])
batch_size = config["models"]["cnn"]["batch_size"]

# Criar data generators
train_gen, val_gen, test_gen = create_data_generators(
    train_df=train_df,
    val_df=val_df,
    test_df=test_df,
    image_size=image_size,
    batch_size=batch_size,
    color_mode='rgb',
    class_mode='categorical',
    augmentation=True
)

# Obter nomes das classes
class_names = list(train_gen.class_indices.keys())
print(f"Classes: {class_names}")
print(f"Índices das classes: {train_gen.class_indices}")

print(f"\nTamanho dos generators:")
print(f"  Treino:    {len(train_gen)} batches")
print(f"  Validação: {len(val_gen)} batches")
print(f"  Teste:     {len(test_gen)} batches")


## 2. Modelo: CNN Simples

Vamos criar e treinar uma CNN construída do zero.


In [ ]:
# Criar modelo
input_shape = (*image_size, 3)  # RGB
num_classes = len(class_names)

model = create_simple_cnn_pneumonia(
    input_shape=input_shape,
    num_classes=num_classes,
    dropout_rate=0.5
)

# Compilar modelo
learning_rate = config["models"]["cnn"]["learning_rate"]
model = compile_model(model, learning_rate=learning_rate)

# Resumo do modelo
print("Arquitetura do Modelo:")
print("=" * 60)
model.summary()


### 2.1 Treinamento do Modelo


In [ ]:
# Configurações de treinamento
epochs = config["models"]["cnn"]["epochs"]
patience = config["models"]["cnn"]["early_stopping_patience"]

# Criar callbacks
checkpoint_path = "../models/pneumonia_cnn_model.h5"
callbacks = get_model_callbacks(
    checkpoint_path=checkpoint_path,
    patience=patience,
    monitor='val_loss'
)

# Treinar modelo
print("Iniciando treinamento...")
print("=" * 60)

history = model.fit(
    train_gen,
    epochs=epochs,
    validation_data=val_gen,
    callbacks=callbacks,
    verbose=1
)

print("\nTreinamento concluído!")


### 2.2 Visualização do Histórico de Treinamento


In [ ]:
# Visualizar histórico de treinamento
plot_training_history(history)


## 3. Avaliação do Modelo

Avaliamos o modelo no conjunto de teste.


In [ ]:
# Carregar melhor modelo (salvo pelo ModelCheckpoint)
best_model = keras.models.load_model(checkpoint_path)

# Avaliar modelo
print("Avaliação no Conjunto de Teste:")
print("=" * 60)
metrics = evaluate_model(best_model, test_gen, class_names, verbose=True)


### 3.1 Matriz de Confusão


In [ ]:
# Obter predições para matriz de confusão
y_pred_proba = best_model.predict(test_gen, verbose=0)
y_pred = np.argmax(y_pred_proba, axis=1)

# Obter labels verdadeiros
y_true = []
for i in range(len(test_gen)):
    _, y_batch = test_gen[i]
    y_true.extend(np.argmax(y_batch, axis=1))
y_true = np.array(y_true)

# Plotar matriz de confusão
plot_confusion_matrix(y_true, y_pred, class_names, normalize=False)
plot_confusion_matrix(y_true, y_pred, class_names, normalize=True)


### 3.2 Curva ROC


In [ ]:
# Plotar curva ROC
y_true_onehot = keras.utils.to_categorical(y_true, len(class_names))
plot_roc_curve(y_true_onehot, y_pred_proba, class_names)


### 3.3 Visualização de Predições

Visualizamos algumas predições corretas e incorretas.


In [ ]:
# Visualizar predições
visualize_predictions(best_model, test_gen, class_names, num_samples=16)


## 4. Interpretabilidade com Grad-CAM

Usamos Grad-CAM para visualizar as regiões da imagem que mais influenciam a predição do modelo.


In [ ]:
# Selecionar algumas amostras para análise Grad-CAM
num_gradcam_samples = 4

# Obter batch do test generator
x_batch, y_batch = test_gen[0]

# Selecionar amostras de cada classe
for class_idx, class_name in enumerate(class_names):
    class_samples = np.where(np.argmax(y_batch, axis=1) == class_idx)[0]
    if len(class_samples) > 0:
        sample_idx = class_samples[0]
        img_array = x_batch[sample_idx]
        
        print(f"\nGrad-CAM para amostra de classe '{class_name}':")
        plot_grad_cam(best_model, img_array, class_names)


## 5. Persistência do Modelo

O modelo já foi salvo automaticamente durante o treinamento pelo ModelCheckpoint. Vamos verificar e fazer uma validação final.


In [ ]:
# Verificar se o modelo foi salvo
import os
if os.path.exists(checkpoint_path):
    print(f"Modelo salvo com sucesso em: {checkpoint_path}")
    
    # Carregar e validar modelo salvo
    loaded_model = keras.models.load_model(checkpoint_path)
    
    # Fazer uma predição de teste
    test_batch_x, test_batch_y = test_gen[0]
    test_pred = loaded_model.predict(test_batch_x[:1], verbose=0)
    test_true = np.argmax(test_batch_y[0])
    test_pred_class = np.argmax(test_pred[0])
    
    print(f"\nValidação do modelo carregado:")
    print(f"  Imagem de teste - Classe verdadeira: {class_names[test_true]}")
    print(f"  Predição: {class_names[test_pred_class]} (confiança: {test_pred[0][test_pred_class]*100:.2f}%)")
    print("\nModelo validado com sucesso!")
else:
    print(f"Modelo não encontrado em: {checkpoint_path}")
    print("Salvando modelo manualmente...")
    best_model.save(checkpoint_path)
    print("Modelo salvo!")
